# Steps to Run Gridded Weather Generator from Phython Environment

**Warning: This is still in progress and may not work**

Prerequisites: 
1. Install latest version of R via https://mirror.lyrahosting.com/CRAN/
2. Create a new python environment with packages rpy2, r-base, r-essentials, pandas, numpy2

Notes (r to python syntax changes)
- replace .(dots) in the parameters with underscores (e.g., nc.path becomes nc_path)
- replace TRUE with True


In [3]:
# Step 1:
#  set all PATH variables

import os
os.environ['PATH'] = 'C:/Program Files/R/R-4.1.2/bin/x64' + os.pathsep + os.environ.get('PATH', '')
os.environ['PYTHONHOME'] = 'C:/Users/taner/Anaconda3/envs/wegentest'
os.environ['PYTHONPATH'] = 'C:/Users/taner/Anaconda3/envs/wegentest/Lib/site-packages'
os.environ['R_HOME'] = 'C:/Program Files/R/R-4.1.2'
os.environ['R_USER'] = 'C:/Users/taner/Anaconda3/envs/wegentest/Lib/site-packages/rpy2'

# Check if variables are correctly defined for rpy2 
import rpy2.situation
for row in rpy2.situation.iter_info():
    print(row)

rpy2 version:
3.4.5
Python version:
3.10.0 | packaged by conda-forge | (default, Nov 20 2021, 02:18:13) [MSC v.1916 64 bit (AMD64)]
Looking for R's HOME:
    Environment variable R_HOME: C:/Program Files/R/R-4.1.2
    InstallPath in the registry: None
    Environment variable R_USER: C:/Users/taner/Anaconda3/envs/wegentest/Lib/site-packages/rpy2
    Environment variable R_LIBS_USER: None
R version:
    In the PATH: 
    Loading R library from rpy2: OK
Additional directories to load R packages from:
None
C extension compilation:


In [4]:
### Import necessary packages
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr

# This is needed for conversion between R and Python syntax
d = {'package.dependencies': 'package_dot_dependencies',
     'package_dependencies': 'package_uscore_dependencies'}

# Load core packages
base = importr('base')
utils = importr('utils')
#utils.chooseCRANmirror(ind=1) # select the first mirror in the list
#devtools = utils.install_packages('devtools')
#devtools = importr('devtools', robject_translations = d)

# Install gridwegen from Github master branch
#gridwegen = devtools.install_github("tanerumit/gridwegen")
gridwegen = importr('gridwegen', robject_translations = d)

In [ ]:
# Path to output files
out_path = "C:/wegentest/ntoum/"
nc_file =  "ntoum.nc"
nc_dimnames = base.list(x = "lon", y = "lat", time = "time")

variables = base.c("precip", "temp", "temp_min", "temp_max")
variable_labels = base.c("Precipitation", "Mean Temperature", "Minimum Temperature", "Maximum Temperature")
variable_units = base.c("mm/day", "DegC", "DegC", "DegC")

# Read-in gridded weather data from netcdf
nc_data_ini = gridwegen.readNetcdf(
    nc_path = base.paste0(out_path, "data/"),
    nc_file = nc_file,
    nc_dimnames = nc_dimnames,
    nc_variables = variables,
    origin_date = base.as_Date("1981-01-01"),
    has_leap_days = True)

# Run simulateWeather function function
gridwegen.simulateWeather(
  climate_data = nc_data_ini[0],
  climate_grid = nc_data_ini[1],
  year_start = 1981,
  year_num = 39,
  month_start = 1,
  variable_names = variables,
  variable_labels = variable_labels,
  variable_units = variable_units,
  sim_year_start = 2020,
  sim_year_num = 40,
  realization_num = 3,
  warm_variable = "precip",
  warm_signif_level = 0.90,
  warm_sample_size = 20000,
  save_warm_results = True,
  knn_annual_sample_size = 100,
  evaluate_model = False,
  evaluate_grid_num = 30,
  apply_delta_changes = False,
  apply_step_changes = True,
  save_scenario_matrix = False,
  output_path = out_path,
  output_ncfile_template = nc_data_ini,
  output_ncfile_prefix = "clim_change_rlz"
)